# PyArrow

- https://towardsdatascience.com/a-gentle-introduction-to-apache-arrow-with-apache-spark-and-pandas-bb19ffe0ddae

In [3]:
import pyarrow.parquet as pq
import os
os.chdir('D:/duckdb/files/RNA2')

In [16]:
%%time
table2 = pq.read_table('anno_2022.parquet')
table2

CPU times: total: 37.2 s
Wall time: 16.6 s


pyarrow.Table
CAR: string
MISURA: string
COD_CE_MISURA: string
BASE_GIURIDICA_NAZIONALE: string
CONCEDENTE: string
COR: string
TITOLO_PROGETTO: string
DESCRIZIONE_PROGETTO: string
CONCESSIONE: timestamp[us]
ATTO_CONCESSIONE: string
IMPRESA: string
CF: string
DES_TIPO_BENEFICIARIO: string
REGIONE: string
ID_COMPONENTE_AIUTO: string
DES_PROCEDIMENTO: string
DESSETTORE_ATTIVITA_OBIETTIVO: string
SETTORE_ATTIVITA: string
DES_STRUMENTO: string
AIUTO: double
NOMINALE: double
----
CAR: [["16781","16781","16781","16781","16781",...,"18945","18945","18945","18945","18945"],["18945","18945","18945","18945","18945",...,"18945","18945","18945","18945","18945"],...,["19754","19754","19754","19754","19754",...,"19754","19754","19754","19754","19754"],["19754","19754","19754","19754","19754",...,"884","884","884","884","884"]]
MISURA: [["FCS - Accordi SNSI e Calcolo ad alte prestazioni","FCS - Accordi SNSI e Calcolo ad alte prestazioni","FCS - Accordi SNSI e Calcolo ad alte prestazioni","FCS - Accord

In [17]:
table2.shape

(6428299, 21)

In [7]:
pq.read_table('anno_2018.parquet', columns=['CAR', 'CONCEDENTE'])

pyarrow.Table
CAR: string
CONCEDENTE: string
----
CAR: [["529","529","529","529","529",...,"526","526","526","526","526"],["526","526","526","526","526",...,"2673","2673","2673","2673","2673"],...,["2255","2255","802","2255","2233",...,"2652","2652","2652","2652","2652"],["2652","2652","2652","2652","2652",...,"4928","1502","3882","1584","884"]]
CONCEDENTE: [["MINISTERO DELLE IMPRESE E DEL MADE IN ITALY - DIREZIONE GENERALE PER GLI INCENTIVI ALLE IMPRESE","MINISTERO DELLE IMPRESE E DEL MADE IN ITALY - DIREZIONE GENERALE PER GLI INCENTIVI ALLE IMPRESE","MINISTERO DELLE IMPRESE E DEL MADE IN ITALY - DIREZIONE GENERALE PER GLI INCENTIVI ALLE IMPRESE","MINISTERO DELLE IMPRESE E DEL MADE IN ITALY - DIREZIONE GENERALE PER GLI INCENTIVI ALLE IMPRESE","MINISTERO DELLE IMPRESE E DEL MADE IN ITALY - DIREZIONE GENERALE PER GLI INCENTIVI ALLE IMPRESE",...,"BANCA DEL MEZZOGIORNO MEDIOCREDITO CENTRALE S.P.A.","BANCA DEL MEZZOGIORNO MEDIOCREDITO CENTRALE S.P.A.","BANCA DEL MEZZOGIORNO MEDIOCREDITO CE

In [11]:
CONCEDENTE = pq.read_pandas('anno_2018.parquet', columns=['CAR','CONCEDENTE']).to_pandas()

In [12]:
CONCEDENTE.describe()

,CAR,CONCEDENTE
count,697091,697091
unique,1473,533
top,526,INPS
freq,129826,210284


In [13]:
CONCEDENTE.sample(10)

,CAR,CONCEDENTE
234513,6048,INPS
198027,884,FONDOPROFESSIONI
353983,2673,MINISTERO DELLE IMPRESE E DEL MADE IN ITALY - ...
41359,2035,VENETO SVILUPPO SPA
413756,526,BANCA DEL MEZZOGIORNO MEDIOCREDITO CENTRALE S....
635320,884,FONDOPROFESSIONI
382931,884,FORMAZIENDA
201694,2334,MINISTERO DELLE IMPRESE E DEL MADE IN ITALY - ...
570191,884,FONDO ARTIGIANATO FORMAZIONE
553115,6047,INPS


In [21]:
%%time
dataset = pq.ParquetDataset(['anno_2018.parquet', 'anno_2019.parquet','anno_2020.parquet', 
                             'anno_2021.parquet', 'anno_2022.parquet','anno_2023.parquet'],
                           columns=["CAR"])
dataset.read_pandas().to_pandas()

TypeError: ParquetDataset.__new__() got an unexpected keyword argument 'columns'

In [20]:
dataset

## Polars

In [32]:
import polars as pl

df = (pl.scan_parquet('anno_2018.parquet').filter(pl.col("CAR").str.contains("203", literal=True)).collect())
df

CAR,MISURA,COD_CE_MISURA,BASE_GIURIDICA_NAZIONALE,CONCEDENTE,COR,TITOLO_PROGETTO,DESCRIZIONE_PROGETTO,CONCESSIONE,ATTO_CONCESSIONE,IMPRESA,CF,DES_TIPO_BENEFICIARIO,REGIONE,ID_COMPONENTE_AIUTO,DES_PROCEDIMENTO,DESSETTORE_ATTIVITA_OBIETTIVO,SETTORE_ATTIVITA,DES_STRUMENTO,AIUTO,NOMINALE
str,str,str,str,str,str,str,str,datetime[ns],str,str,str,str,str,str,str,str,str,str,f64,f64
"""2033""","""[Titolo II - c…","""SA_39759""","""Regolamento de…","""REGIONE PUGLIA…","""220329""","""GRANORO 2015""","""Ampliamento de…",2018-01-30 00:00:00,"""110""","""PASTIFICIO ATT…","""03379290723""","""Grande impresa…","""Puglia""","""237574""","""Esenzione""","""Aiuti a finali…","""C.10.7""","""Sovvenzione/Co…",2.9100e6,2.9100e6
"""2033""","""[Titolo II - c…","""SA_39759""","""Regolamento de…","""REGIONE PUGLIA…","""221586""","""STUDIO DENTIST…","""Il programma d…",2018-01-30 00:00:00,"""113""","""STUDIO DENTIST…","""04220880753""","""PMI""","""Puglia""","""114269""","""Esenzione""","""Aiuti a finali…","""Q.86.2""","""Sovvenzione/Co…",183296.5,183296.5
"""2033""","""[Titolo II - c…","""SA_39759""","""Regolamento de…","""REGIONE PUGLIA…","""221574""","""POLOGELATI S.R…","""La società Pol…",2018-01-31 00:00:00,"""133""",""""" POLOGELATI …","""03196220713""","""PMI""","""Puglia""","""114511""","""Esenzione""","""Aiuti a finali…","""C.10.5""","""Sovvenzione/Co…",160492.62,160492.62
"""2033""","""[Titolo II - c…","""SA_39759""","""Regolamento de…","""REGIONE PUGLIA…","""203248""","""MODI - SOCIETA…","""La società pro…",2018-01-15 00:00:00,"""5""","""MODI - SOCIETA…","""07599370728""","""PMI""","""Puglia""","""124665""","""Esenzione""","""Aiuti a finali…","""I.56.1""","""Sovvenzione/Co…",54757.32,54757.32
"""2033""","""[Titolo II - c…","""SA_39759""","""Regolamento de…","""REGIONE PUGLIA…","""208066""","""PGH BARONE DI …","""Il programma d…",2018-01-16 00:00:00,"""A.D. N. 7""","""PGH BARONE DI …","""04819280753""","""PMI""","""Puglia""","""165607""","""Esenzione""","""Aiuti a finali…","""I.55.1""","""Sovvenzione/Co…",3.9659e6,3.9659e6
"""203""","""Nuova Sabatini…","""SA.40429""","""Decreto Legge …","""MINISTERO DELL…","""193511""","""Agevolazione p…","""Agevolazione p…",2018-01-18 00:00:00,"""825""","""AUTOTRASPORTI …","""01964840209""","""PMI""","""Lombardia""","""211964""","""Esenzione""","""Aiuti agli inv…","""H.49.4""","""Sovvenzione/Co…",7564.55,7564.55
"""203""","""Nuova Sabatini…","""SA.40429""","""Decreto Legge …","""MINISTERO DELL…","""193512""","""Agevolazione p…","""Agevolazione p…",2018-01-18 00:00:00,"""826""","""AUTOTRASPORTI …","""00212670020""","""PMI""","""Piemonte""","""211965""","""Esenzione""","""Aiuti agli inv…","""H.49.4""","""Sovvenzione/Co…",6559.76,6559.76
"""203""","""Nuova Sabatini…","""SA.40429""","""Decreto Legge …","""MINISTERO DELL…","""193514""","""Agevolazione p…","""Agevolazione p…",2018-01-18 00:00:00,"""829""","""SACCHETTO COST…","""10491040019""","""PMI""","""Piemonte""","""211967""","""Esenzione""","""Aiuti agli inv…","""F.43.3""","""Sovvenzione/Co…",4090.2,4090.2
"""203""","""Nuova Sabatini…","""SA.40429""","""Decreto Legge …","""MINISTERO DELL…","""193515""","""Agevolazione p…","""Agevolazione p…",2018-01-18 00:00:00,"""830""","""ILVE S.R.L.""","""02239981208""","""PMI""","""Emilia-Romagna…","""211968""","""Esenzione""","""Aiuti agli inv…","""C.28.2""","""Sovvenzione/Co…",8234.43,8234.43


## PyArrow

In [8]:
import pyarrow.parquet as pq
import pandas as pd
import duckdb

con = duckdb.connect()
con.execute("PRAGMA threads=4") # enable automatic query parallelization
con.execute("PRAGMA enable_object_cache") # enable caching of parquet metadata

In [9]:
%%time
print(con.execute("SELECT COUNT(*) FROM 'anno*.parquet'").df())

   count_star()
0      16395093
CPU times: total: 31.2 ms
Wall time: 120 ms


## ricerca

In [6]:
df = con.execute("SELECT CF, impresa FROM 'anno*.parquet' group by 1,2").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [7]:
df.shape

(4971170, 2)

In [31]:
%%time
df = (con.execute("SELECT CAR, CONCEDENTE, MISURA,CF,nominale,aiuto FROM 'anno*.parquet';").df())

CPU times: total: 15.9 s
Wall time: 20.3 s


In [11]:
df = df.replace({'CONCEDENTE' : { 
    'AGENZIA ENTRATE':'AGENZIA DELLE ENTRATE', 
    'AGENZIE DELLE ENTRATE':'AGENZIA DELLE ENTRATE', 
    'BANCA DEL MEZZOGIORNO MEDIOCREDITO CENTRALE S.P.A QUALE RAPPRESENTANTE E MANDATARIA DEL RTI COMPOSTO DA MEDIOCREDITO CENTRALE - INVITALIA E ARTIGIANCASSA S.P.A.':'BANCA DEL MEZZOGIORNO MEDIOCREDITO CENTRALE S.P.A.',
    "INVITALIA - AGENZIA NAZIONALE PER L'ATTRAZIONE DEGLI INVESTIMENTI E LO SVILUPPO D'IMPRESA S.P.A.":"INVITALIA",
    "INVITALIA AGENZIA NAZIONALE PER L'ATTRAZIONE DEGLI INVESTIMENTI E LO SVILUPPO D'IMPRESA S.P.A.":"INVITALIA",
    "INVITALIA_ AGENZIA NAZIONALE PER L'ATTRAZIONE DEGLI INVESTIMENTI E LO SVILUPPO DI IMPRESA S.P.A.":"INVITALIA",
    "AGENZIA NAZIONALE PER L'ATTRAZIONE DEGLI INVESTIMENTI E LO SVILUPPO D'IMPRESA S.P.A. - INVITALIA (05678721001)":"INVITALIA",
    "AGENZIA NAZIONALE PER L'ATTRAZIONE DEGLI INVESTIMENTI E LO SVILUPPO D'IMPRESA SPA - INVITALIA (05678721001)":"INVITALIA",
    "AGENZIA NAZIONALE PER L'ATTRAZIONE DEGLI INVESTIMENTI E LO SVILUPPO D'IMPRESA SPA- INVITALIA SPA":"INVITALIA",
    "INVITALIA S.P.A":"INVITALIA",
}})


In [ ]:
misure = (con.execute("SELECT CAR, MISURA FROM 'anno*.parquet';").df())

In [ ]:
## rimuovere CAR
- MCC
- Covid


## raggruppare

In [30]:
df.groupby(['CAR','MISURA'])['CF'].count().sort_values(ascending=False).head(20)

KeyError: 'CAR'

In [17]:
df.sort_values(['CF','count'],ascending=False).groupby('MISURA').head(20)

KeyError: 'count'

In [21]:
df.groupby(['CONCEDENTE','MISURA']).agg({'NOMINALE':sum}).sort_values(['NOMINALE'],ascending=False)

C:\Users\PVolterr\AppData\Local\Temp\ipykernel_14516\2972013851.py:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'sum' instead.
  df.groupby(['CONCEDENTE','MISURA']).agg({'NOMINALE':sum}).sort_values(['NOMINALE'],ascending=False)


NOMINALE
CONCEDENTE                                         MISURA                                                          
BANCA DEL MEZZOGIORNO MEDIOCREDITO CENTRALE S.P.A. COVID-19:  Fondo di garanzia PMI Aiuto di stato...  9.233889e+10
                                                   Fondo di garanzia per le piccole e medie imprese    8.981360e+10
SACE S.P.A.                                        Garanzia Italia                                     4.235417e+10
BANCA DEL MEZZOGIORNO MEDIOCREDITO CENTRALE S.P.A. COVID-19: Fondo di garanzia PMI - Modifica SA.5...  3.979434e+10
                                                   COVID-19:  Fondo di garanzia PMI Aiuto di stato...  2.885533e+10
...                                                                                                             ...
CAMERA DI COMMERCIO INDUSTRIA ARTIGIANATO E AGR... FINANZIAMENTO PER LA REALIZZAZIONE DI DIAGNOSI ...  0.000000e+00
REGIONE LOMBARDIA - DG SVILUPPO ECONOMICO          PROROGA E AGGIORNAMENTO DEL REGIME QUADRO REGIO...  0.000000e+00
                                                   BANDO PER L'ASSEGNAZIONE DI CONTRIBUTI ALLE IMP...  0.000000e+00
COMUNE DI PALMANOVA                                MISURE PER RISTORARE LE IMPRESE ESERCENTI TRASP...  0.000000e+00
COMUNE DI PIETRAPORZIO                             Regime quadro nazionale sugli aiuti di Stato – ...  0.000000e+00

[14494 rows x 1 columns]

In [27]:
grouped = df.groupby('MISURA', sort=False).count()
grouped.sort_index(ascending=False)
print(grouped)

                                                    CONCEDENTE     CF  \
MISURA                                                                  
FCS - Accordi SNSI e Calcolo ad alte prestazioni           729    729   
Sostegno del Fondo per la crescita sostenibile ...         145    145   
Smart&Start - Riordino degli interventi di sost...        1149   1149   
NIT0 - Nuove imprese a tasso zero                         2730   2730   
Resto al Sud                                             31717  31717   
...                                                        ...    ...   
BANDO PER LA CONCESSIONE DI CONTRIBUTI PER IL C...           9      9   
Bando voucher digitali I4.0 per le PMI della Pr...          22     22   
RER - LEGGE REGIONALE 23 GIUGNO 2017 N. 11 - "S...           7      7   
RER - DGR 718/2018 - POR FESR 2014 - 2020 - ASS...         293    293   
RER - LR 26/2009 Bando 2018 per la concessione ...          12     12   

                                                  

In [37]:
df[['CAR','CF']].groupby(['CAR'])['CF'].count().nlargest(5)

CAR
19754    2986302
15959    2207400
12627    1422317
20177     906830
15884     894935
Name: CF, dtype: int64

In [36]:
MCC = df[(df['CONCEDENTE'] == 'BANCA DEL MEZZOGIORNO MEDIOCREDITO CENTRALE S.P.A.')]
MCC.groupby(['CAR','MISURA']).count()

,,CONCEDENTE,CF,NOMINALE,AIUTO
CAR,MISURA,,,,
12627,COVID-19: Fondo di garanzia PMI Aiuto di stato SA. 56966 (2020/N),1422317,1422317,1422317,1422317
15884,GARANZIA DEL FONDO A VALERE SULLA SEZIONE SPECIALE DI CUI ALL’ARTICOLO 56 DEL DECRETO-LEGGE DEL 17 MARZO 2020 N. 18,894935,894935,894935,894935
16615,COVID-19: Fondo di garanzia PMI Aiuto di stato SA.59655 - Proroga SA.56966,390172,390172,390172,390172
16617,Fondo di garanzia per le piccole e medie imprese,422443,422443,422443,422443
18730,"COVID-19: Fondo di garanzia PMI - Modifica SA.56966, SA.57625, SA.59655",440864,440864,440864,440864
23546,TCF: Garanzie sui prestiti per PMI e piccole imprese a media capitalizzazione,28930,28930,28930,28930
24646,TCF: Garanzie sui prestiti per PMI e piccole imprese a media capitalizzazione,108269,108269,108269,108269
24793,GARANZIA DEL FONDO A VALERE SULLA SEZIONE SPECIALE DI CUI ALL’ARTICOLO 56 DEL DECRETO-LEGGE DEL 17 MARZO 2020 N. 18,6,6,6,6
27308,Fondo di garanzia per le piccole e medie imprese,34147,34147,34147,34147


In [35]:
INVITALIA = df[(df['CONCEDENTE'] == 'INVITALIA')]
INVITALIA.groupby(['CAR','MISURA']).count()

,,CONCEDENTE,CF,NOMINALE,AIUTO
CAR,MISURA,,,,
19845,"Interventi a favore delle micro, piccole e medie imprese nelle zone colpite dagli eventi sismici",34,34,34,34
24246,Programma unitario di intervento - interventi per le aree del terremoto del 2009 e 2016.,855,855,855,855
24260,"Programma unitario di intervento - interventi per le aree del terremoto 2009-2016 - B1.3.b. ""Interventi per l'innovazione diffusa""",319,319,319,319
24734,"Programma unitario di intervento - Interventi per le aree del terremoto del 2009 e 2016 - B.1.3.a Interventi per la nascita, lo sviluppo e il consolidamento di iniziative micro-imprenditoriali e per l’attrazione e il rientro di imprenditori",3,3,3,3
24756,Economia circolare e filiere agroalimentari,2,2,2,2
24824,"Programma unitario di intervento - Interventi per le aree del terremoto del 2009 e 2016 - B1.3.c “Interventi per l’avvio, il riavvio e il consolidamento di attività economiche e per il rientro di quelle temporaneamente delocalizzate”",16,16,16,16
24971,B1.1 “Sostegno agli investimenti di rilevante dimensione finanziaria”- Contratto di sviluppo Sisma,2,2,2,2
25001,"Programma unitario di intervento - Interventi per le aree del terremoto del 2009 e 2016 – B1.2 “Interventi per progettualità di dimensione intermedia” e B.3.3 “Ciclo delle macerie"".",45,45,45,45


In [34]:
AdE = df[(df['CONCEDENTE'] == 'AGENZIA DELLE ENTRATE')]
pd.set_option('display.max_rows', None)
AdE.groupby(['CAR','MISURA']).count()

,,CONCEDENTE,CF,NOMINALE,AIUTO
CAR,MISURA,,,,
10373,"Credito d'imposta sugli investimenti pubblicitari incrementali su quotidiani, periodici e sulle emittenti televisive e radiofoniche locali",40893,40893,40893,40893
12190,ABRUZZO - IRAP AGEVOLATA FARMACIE RURALI,358,358,358,358
12208,ABRUZZO - IRAP AGEVOLATA COMUNI MONTANI FINO A 3.000 ABITANTI,602,602,602,602
12209,ABRUZZO - IRAP AGEVOLATA ONLUS E COOPERATIVE SOCIALI,579,579,579,579
12210,ABRUZZO - IRAP AGEVOLATA NUOVE INIZIATIVE (PRIMO BIENNIO ATTIVITA'),91,91,91,91
12469,"Detassazione di contributi, indennizzi e risarcimenti",812,812,812,812
13138,credito di imposta per gli investimenti nelle Zone economiche speciali,1015,1015,1015,1015
13139,Esenzione per i soggetti passivi che intraprendono stabilmente nuove iniziative economiche nel territorio regionale dal periodo di imposta in corso al 1 gennaio 2019 e fino al periodo di imposta in corso al 1 gennaio 2021. L'esenzione si applica per i pri,27,27,27,27
13140,"Esenzione per gli Enti cooperativi a mutualita' prevalente, iscritti nel registro regionale degli Enti cooperativi di cui all'art. 3, della L.R. 27/98, soggetti all'aliquota di cui all'articolo 16, comma 1 del D.Lgs. 446/1997",64,64,64,64


In [98]:
AdE.sample()

,CONCEDENTE,MISURA,CF,NOMINALE,AIUTO
9431713,AGENZIA DELLE ENTRATE,esenzioni fiscali e crediti d'imposta adottati...,TNDMNL78A22D862N,91.0,91.0


In [99]:
import duckdb
import os
os.chdir('D:')
conn = duckdb.connect(database='RNA.duckdb', read_only=False)
conn.execute(f'''CREATE TABLE RNA AS SELECT * FROM read_parquet('D:/duckdb/files/RNA2/anno*.parquet');''').df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Count
0,16395093
